In [ ]:
import sys, os
sys.path.append('../../dependencies/')
import mfsetup
import flopy as fp
import flopy.utils as fu
from flopy.utils.gridintersect import GridIntersect

import shapefile as sf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from shapely.geometry import Point, LineString
import geopandas as gp
datapath = Path('../../data/sgn/')

## one-liner to build the model

In [ ]:
# note that modflow setup changes the directory, so good idea to change back at the end
stdir = os.getcwd()

In [ ]:
m = mfsetup.MF6model.setup_from_yaml('sgn_config.yml')

In [ ]:
m.write_input()

### set up an intersection object

In [ ]:
ix = GridIntersect(m.modelgrid)

## need to add a GHB package - manually for now

In [ ]:
# read in the boundary polygon
boundary = gp.read_file(datapath / 'shp' / 'Model_domain.shp')

# create a polyline from the polygon
boundary_line =  LineString([Point(i) for i in boundary.iloc[0].geometry.exterior.coords])
boundary_line

# instersect the bounding polygon with the grid to get boundary cells
ix_boundary = ix.intersect(boundary_line.buffer(25, resolution=4))

In [ ]:
ix_df = pd.DataFrame.from_records(ix_boundary)
ix_df.head()

In [ ]:
# read in the september 2019 measured heads to make the GHB data
sep19_head = fu.Raster.load(datapath / 'raster' / 'heads_sep2019.asc')
plt.imshow(sep19_head.get_array(1))
plt.colorbar()

In [ ]:
# resample these heads to the model
head = sep19_head.resample_to_grid(m.modelgrid,band=sep19_head.bands[0], method="nearest")

In [ ]:
# make the GHB stress period data
ghb_spd = []
for icpl in ix_boundary.cellids:
    cond = 100.0
    for layer in [0,2]:
        # note - structured grid so *icpl unpacks row/col
        ghb_spd.append([(layer, *icpl), head[icpl], cond, f'GHB{layer+1}'])
       

In [ ]:
# create the package
ghb = fp.mf6.ModflowGwfghb(model=m, 
                                stress_period_data=ghb_spd, 
                                boundnames=True) 

## also need to add potentially flowing multi-aquifer wells manually for now 

In [ ]:
# read in the zero-q wells
maw_wells = pd.read_csv(datapath / 'wells_zero.csv', index_col = 0)
maw_wells.head()

### reset the index for MAW wells to be sequential only among the remaining wells

In [ ]:
# get the row/column location for each well
maw_wells['cellid']=[ix.intersect(Point(x,y))[0][0] for x,y in zip(maw_wells.x,maw_wells.y)]
maw_wells.head()

In [ ]:
# also get the top, bottom, and starting head of each cell containing a well
maw_wells['top'] = [m.dis.top.array[r,c] for r,c in maw_wells.cellid]
maw_wells['bot'] = [m.dis.botm.array[-1,r,c] for r,c in maw_wells.cellid]
maw_wells['strt'] = [m.ic.strt.array[0,r,c] for r,c in maw_wells.cellid]


## now make the package data
## this takes the following format
`packagedata : [wellno, radius, bottom, strt, condeqn, ngwfnodes, aux,  boundname]`
where:
* `wellno` is just a sequential number used later to identify the wells
* `radius` is the radius in model units (assuming 10cm for now)
* `bottom` is the bottom elevation of the well
    * note: for `bottom` we are setting the wells to span the entire aquifer
* `strt` is the starting head for the well (using the model starting head)
* `condeq` is how conductance is calculated (we will use THIEM)
* `ngwfnodes` is the number of nodes the well will be connected to. In our case, this is 3 - one for each layer that the cell is in
* `aux` is the number of auxiliary values - we will have 0
* `boundname` is a name for the well. We will use the name from the original data

In [ ]:
maw_radius = 0.1

In [ ]:
maw_packagedata = []
for wellno,cwell in maw_wells.iterrows():
    maw_packagedata.append([
        wellno, maw_radius, cwell.bot, cwell.strt, 'THIEM', 3
    ])

## now we need to set up the connection data
## this take the following format
`connectiondata : [wellno, icon, cellid, scrn_top, scrn_bot, hk_skin, radius_skin]`
where:
* `wellno` is the well number, as defined above
* `icon` is the number of connections (3 in our case)
* `cellid` is 3 integers indicating layer, row, and column
* `scrn_top` is the top of the screen (the top of the model in our case)
* `scrn_bot` is the bottom of the screen (the bottom of the model in our case)
    * note: the previous elevations are ignored when THIEM is used to calculate conductance as MODFLOW6 uses the top and bottom of the cell automagically
* `hk_skin` and `radius_skin` are also ignored when THIEM is used to calculate conductance


In [ ]:
maw_connections = []
for wellno,cwell in maw_wells.iterrows():
    for clay in range(3):
        maw_connections.append([
            wellno, clay, clay, cwell.cellid[0], cwell.cellid[1], cwell.top, cwell.bot, -999, -999
        ])


In [ ]:
# explicitly set all pumping rates to 0
maw_spd = [[i, 'rate', 0] for i in range(len(maw_wells))]

### now set the MAW package into the model if requested

In [ ]:
include_MAW = False

In [ ]:
if include_MAW:
    maw = fp.mf6.ModflowGwfmaw(
            model=m,
            no_well_storage=True,
            nmawwells=len(maw_wells),
            packagedata=maw_packagedata,
            connectiondata=maw_connections,
            perioddata=maw_spd,
        )
    # set up some observations
    obs_file = f"{m.name}.maw.obs"
    csv_file = f"{obs_file}.csv"
    head_obs =  [['head','head', i+1] for i in range(len(maw_wells))]
    Qobs = []
    for i in range(len(maw_wells)):
        Qobs.extend([[f'Q.{i}_lay{j}', 'maw', i+1, j+1 ] for j in range(3)])
    obs_dict = {
        csv_file: 
        head_obs + Qobs
    }
    maw.obs.initialize(filename = obs_file, digits  = 8, print_input=True, continuous=obs_dict)

In [ ]:
m.write_input()

In [ ]:
m.name

In [ ]:
m.dis.export('testgrid.shp')

## quick hack on IMS to turn off PTC

In [ ]:
ims = [i.rstrip() for i in open('../../models/sgn_mfsetup/sgn_50_sim.ims').readlines()]

In [ ]:
with open('../../models/sgn_mfsetup/sgn_50_sim.ims', 'w') as ofp:
    [ofp.write("{}\n".format(line.replace('END options', '  NO_PTC ALL\nEND options'))) for line in ims]

In [ ]:
os.system('mf6')

In [ ]:
top = m.dis.top.array

In [ ]:
plt.imshow(top)

In [ ]:
h = fu.binaryfile.HeadFile('../../models/sgn_mfsetup/sgn_50.hds').get_alldata()

In [ ]:
h.shape

In [ ]:
plt.imshow(np.squeeze(h)[0][0])
plt.colorbar()

In [ ]:
plt.imshow(np.squeeze(h)[1][0])
plt.colorbar()

### make sure the barrier well shows up in the secondx stress period

In [ ]:
plt.imshow(np.squeeze(h)[1][0]-np.squeeze(h)[0][0])
plt.colorbar()

### check flooding

In [ ]:
flooded = top-np.squeeze(h)[0][0]
plt.imshow(flooded)
plt.colorbar()

In [ ]:
flooded[flooded>0] = np.nan
plt.imshow(flooded)
plt.colorbar()

In [ ]:
df1,df2 = fu.Mf6ListBudget('../../models/sgn_mfsetup/sgn_50.list').get_dataframes(start_datetime='2020-12-31')

In [ ]:
df1

In [ ]:
df2

## check out the MAW obs

In [ ]:
if include_MAW:
    maw_df = pd.read_csv('../../models/sgn_mfsetup/sgn_50.maw.obs.csv', index_col=0)

In [ ]:
maw_df